In [ ]:
#-configuration---------
transmissions = [0.005:0.005:1.0]
temp = 1e-7
integration = (-15.0, 15.0)
voltage = (0.005, 0.005, 6.00)
#-----------------------
num_workers = 10
;

In [2]:
#-checks----------------
if temp <= 0
    println("Temperature is 0.\n Assuming default value of 1e-7")
    temp = 1e-7
end

@assert voltage[1] >= 0.005 "Voltage must be greater or equal to 0.005"
#----------------------

In [3]:
#-dependencies----------
using ProgressMeter
using Plots
using JLD2
using Distributed
#----------------------

In [4]:
#-Constants------------
const G₀ = 7.748091729e-5
const e = 1.602176634e-19
const h = 6.62607015e-34
#----------------------
;

In [5]:
# convert input parameters
transmissions = vcat([collect(t) for t in transmissions]...)
voltages = collect(voltage)

# find working dirs
start_dir = @__DIR__
temp_dir = joinpath(@__DIR__, "..", "temp")
bin_dir = joinpath(@__DIR__, "..", "bin")
plots_dir = joinpath(@__DIR__, "..", "plots")
data_dir = joinpath(@__DIR__, "..", "data")
;

In [6]:
cd(temp_dir)
for i in 1:num_workers
   run(`mkdir -p thread_$i`)
   run(`cp $bin_dir/iv $temp_dir/thread_$i`)
   open(joinpath(temp_dir,"thread_$i", "do.sh"), "w") do file
      write(file, "cat temp_config.txt | ./iv")
   end
   run(`chmod +x $temp_dir/thread_$i/do.sh`)
end

In [7]:
addprocs(num_workers);

In [8]:
@everywhere using ProgressMeter
@everywhere begin
    using Serialization  # For sending complex data types

    function run_simulation(i, t, temp, integration, voltage, temp_dir)
        thread_dir = joinpath(temp_dir, "thread_$(myid()-1)")

        run(`rm -f $(joinpath(thread_dir, "iv.dat"))`)
        run(`rm -f $(joinpath(thread_dir, "temp_config.txt"))`)

        open(joinpath(thread_dir, "temp_config.txt"), "w") do file
            write(file, "  $t  $temp\n")
            write(file, "$(integration[1])  $(integration[2])\n")
            write(file, "  $(voltage[1])  $(voltage[3])  $(voltage[2])\n")
        end

        run(`bash -c "cd $(thread_dir) && ./do.sh"`)

        x, y = Float64[], Float64[]

        try
            open(joinpath(thread_dir, "iv.dat"), "r") do file
                for line in eachline(file)
                    parts = split(strip(line))
                    if length(parts) >= 2
                        push!(x, parse(Float64, parts[1]))
                        push!(y, parse(Float64, parts[2]))
                    end
                end
            end
        catch e
            println("Error processing file for t=$t: $e")
        end

        return (t, (x, y))
    end
end

In [ ]:
curves = Dict{Float64, Tuple{Vector{Float64}, Vector{Float64}}}()
args_list = [(i, transmissions[i], temp, integration, voltage, temp_dir) for i in 1:length(transmissions)]
results = @showprogress 1 "Working..." pmap(x -> run_simulation(x...), args_list)

for (t, data) in results
    curves[t] = data
end

const raw = copy(curves);

In [ ]:
curve_processed = Dict{Float64,Tuple{Vector{Float64},Vector{Float64}}}()
for (τ, curve) in curves
    x, y = curve
    x = copy(x)
    y = copy(y)
    G = G₀ * τ
    y .*= (2*e^2 / h)
    y ./= G
    curve_processed[τ] = (x, y)
end
const normalized = copy(curve_processed);

In [ ]:
curve_derived = Dict{Float64,Tuple{Vector{Float64},Vector{Float64}}}()
for (τ, curve) in curve_processed
    x, y = curve
    y = diff(y) ./ diff(x)
    x = x[1:end-1]
    curve_derived[τ] = (x, y)
end
const derived = copy(curve_derived);

In [ ]:
curve_d2 = Dict{Float64, Tuple{Vector{Float64}, Vector{Float64}}}()
for (τ, curve) in derived
    x, y = curve
    y = diff(y) ./ diff(x)
    x = x[1:end-1]
    curve_d2[τ] = (x, y)
end

In [ ]:
jldopen(joinpath(data_dir, "iv.jld2"), "w") do file
    file["raw"] = raw
    file["normalized"] = normalized
    file["derived"] = derived
end;

In [ ]:
using CairoMakie
using ColorSchemes

t_values = sort(collect(keys(normalized)))

max_clip = 1.0  
min_clip = 0.35

clip_normalize(t, min_val, max_val) = t_rel = min_val + t * (max_val - min_val)

see = get(ColorSchemes.Blues, range(min_clip, max_clip, length=256))

fig = Figure()
ax = Axis(fig[1, 1])

for (i, t) in enumerate(t_values)
    x, y = normalized[t]
    color_idx = clip_normalize(t, min_clip, max_clip)
    lines!(ax, x, y, color=cgrad(:Blues)[color_idx], linewidth=2)
end
ax.xlabel = "eV₀ (Δ)"
ax.ylabel = "Normalized Current (GΔ/e)"
ax.limits = (0, 2.5), (0, 5)

Colorbar(fig[1, 2], colormap=see, ticks=0:0.2:1, label="Transmission (τ)")

display(fig)
save(joinpath(plots_dir, "iv.svg"), fig)

In [ ]:
fig = Figure()
ax = Axis(fig[1, 1])

for (τ, curve) in sort(collect(derived), by = x -> x[1])
    x, y = curve
    color_idx = clip_normalize(τ, min_clip, max_clip)
    lines!(ax, x, y, color=cgrad(:Blues)[color_idx], linewidth=2)
end
ax.xlabel = "eV₀ (Δ)"
ax.ylabel = "dI/dV₀ (G)"
ax.limits = (0, 2.5), (0, 40)

Colorbar(fig[1, 2], colormap=see, ticks=0:0.2:1, label="Transmission (τ)")
display(fig)
save(joinpath(plots_dir, "didv.svg"), fig)